In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy import linalg
import matplotlib.pyplot as plt
from datetime import datetime
import random
from IPython.display import display, clear_output
from ipywidgets import widgets, Layout, interact, interactive, fixed, interact_manual
from jupyterthemes import jtplot
import time
import scipy as scipy
from scipy import stats
import csv


In [2]:
def post_process(CRN_output, He3_output):
    ERs_all = He3_output[2]
    export_inputs()
    
    He3_percent_mat = np.empty((total_time, scenarios))
    
    for i in range(scenarios):
        He3_percent_mat[:,i] = np.divide(He3_output[1][:,i], CRN_output[1][:,1,i]) * P0_Be10 / P0_He3

    master_array = np.empty((total_time, 11, scenarios))

    for i in range(scenarios):
        master_array[:,:3,i] = CRN_output[1][:,:,i]
        master_array[:,3,i] = He3_output[1][:,i]
        master_array[:,4,i] = He3_percent_mat[:,i]
        master_array[:,5,i] = rockfall_output[0][:,i]
        master_array[:,6:,i] = comparison_constants[:,:,i]
        
        np.savetxt('D:/STEIN_paper/model_outputs/stochastic_erosion/samples_output_' + str(ERs_all[i]) + '.csv',
               samples[:,:,i],
               delimiter = ',')
        
        np.savetxt('D:/STEIN_paper/model_outputs/stochastic_erosion/comparison_output_' + str(ERs_all[i]) + '.csv',  
               master_array[:,:,i],
               delimiter = ',')
        
    np.savetxt('D:/STEIN_paper/model_outputs/stochastic_erosion/measured_output_' + str(np.mean(ERs_all)) + '.csv',
              measured,
              delimiter = ',')
    
    master_means = [[np.mean(master_array[2500:,i,j]) for j in range(scenarios)] for i in range(11)]
    
    np.savetxt('D:/STEIN_paper/model_outputs/stochastic_erosion/all_year_means_output_' + str(np.mean(ERs_all)) + '.csv',
              master_means,
              delimiter = ',')
    
    master_df = [pd.DataFrame(data = master_array[:,:,i],
                index = np.arange(0,total_time,1),
                columns = ['C-14 Conc. Stoch', 
                           'Be-10 Conc. Stoch', 
                           'C-14/Be-10 Ratio Stoch',
                           'He-3 Conc. Stoch',
                           'Perc. He-3 Stoch',
                           'Rockfall Size',
                           'C-14 Conc. Constant',
                           'Be-10 Conc. Constant',
                           'C-14/Be-10 Ratio Constant',
                           'He-3Conc. Constant',
                           'Perc. He-3 Constant']) for i in range(scenarios)]
   
    return master_array, master_df, master_means

def export_inputs():
    input_export = ['Erosion Type: ', str(erosion_type), '\n', 
                     '\n Total Time: ', str(total_time),
                     '\n SLHL C14 P0: ', str(SLHL_C14), 
                     '\n SLHL Be10 P0: ', str(SLHL_Be10),
                     '\n SLHL He3 P0: ', str(SLHL_He3), 
                     '\n CN Scaling Factor: ', str(scaling_factor), 
                     '\n Grain Size: ', str(gr_sz),
                     '\n Mean Annual T (C): ', str(MAT),
                     '\n Seasonal Temperature Amplitude (C): ', str(T_AMP),
                     '\n Number of Scenarios: ', str(scenarios),
                     '\n Base (Mean) Erosion Rate: ', str(stoch_base_ER), 
                     '\n Number of Samples: ', str(sample_no),
                     '\n Max Depth for Calculations: ', str(max_depth), 
                     '\n Rock Density (g cm3) ', str(rock_rho), 
                     '\n Cosmogenic Nuclide Atten. Length: ', str(CN_lambda),
                     '\n Pareto Distribution Parameters: ', str(Pareto_params)]
                     # '\n Inverse Gaussian Distribution Parameters: ', str(Inv_Gauss_params)]
    var_export = open('D:/STEIN_paper/model_outputs/stochastic_erosion/STEIN_variables_STO_' + str(np.mean(measured_ER)) + '_' + str(time.strftime('%d-%m-%Y_%H.%M')[:18]) + '.txt', 'w')   
    var_export.writelines(input_export)
    var_export.close()

def write_var_export(i):
    text_file = open("Output_" + str(datetime.now().strftime("%m_%d_%Y_%H_%M_%S")) + '_' + str(erosion_type) + 
                     str(rockfall_output[1][0]) + '.txt', "w")
    
    variable_export = pd.DataFrame(data = None, index = None, columns = None)
    
    variable_export.loc[:,0] = [erosion_type, total_time, SLHL_C14 , SLHL_Be10,
                            SLHL_He3, rockfall_output[1][:], 
                            scaling_factor, gr_sz, MAT, T_AMP, save_output,
                            scenarios, stoch_base_ER, sample_no, max_depth, rock_rho,
                            CN_lambda, mu, dt, rho, nx, r, dx, alpha, P0_Be10, 
                            P0_He3, save_output, shift_ER, save_only_surf, time_ER_shift, Pareto_params[i]]
    
    variable_export.index = ['Erosion Type', 'Total Time', 'SLHL C14 P0', 'SLHL Be10 P0', 
                            'SLHL He3 P0', 'True Erosion Rates (cm yr$^{-1}$)', 'Scaling Factor', 
                        'Grain Size (cm)', 'Mean Annual Temperature', 'Seasonal Temp. Amplitude',
                         'Export Output [T/F]', 'Number of Scenarios (stochastic only)', 
                         'Base Erosion Rate (stochastic only)', 'Number of Sub-samples for cosmo',
                        'Maximum Depth of Model [He : Be/C]', 'Rock Density', 'CN Lambda', 'CN mu',
                        'dt (He)', 'Quartz Density (He)', 'Qtz Nodes (He)', 'Quartz Grain Size (cm)' ,
                         'Qtz Node Spacing (He)', 'Diffusivity (Qtz)', 'Be10 P0 (scaled)', 'He3 P0 (scaled)', 'Save Output?',
                        'Change Er. Rate mid-run?', 'Save Only He3 Mean Surf. Conc?', 'Year of Er. Rate shift',
                            'Pareto Distribution Factors [x | Shape | Scale]']
    variable_export.to_csv('Output_' + str(datetime.now().strftime("%m_%d_%Y_%H_%M_%S")) + '_' + str(erosion_type) + 
                     str(rockfall_output[1][0]) + '.txt', index_label = 'Variable' )

In [3]:
erosion_type = 'stochastic' 
total_time = 17500
SLHL_C14 = 12.76
SLHL_Be10 = 4.02
SLHL_He3 = 128.0
scaling_factor = 1
gr_sz = 0.1
MAT = -5.0
T_AMP = 5.0
save_output = True
scenarios = 10
stoch_base_ER = 0.1
sample_no = 30
max_depth = [350, 1500]
rock_rho = 2.6
CN_lambda = 160
mu = rock_rho / CN_lambda
dt = 1

# He diffusion inputs
rho = 2.65 # quartz density
nx = 100
r = gr_sz
dx = (r / nx)
x = np.arange(dx/2, r, dx)
x_up = x + (dx/2)
x_low = x - (dx/2)
x_vol = [(4/3 * np.pi * x_up[i]**3) - (4/3 * np.pi * x_low[i]**3) for i in range(len(x))]
shell_mass = [(x_vol[i] * rho) for i in range(len(x_vol))]
alpha = 1.5e-6

P0_Be10 = SLHL_Be10 * scaling_factor
P0_He3 = SLHL_He3 * scaling_factor

save_output = True
shift_ER = False
save_only_surf = True
time_ER_shift = 23000

In [4]:
Pareto_params = [['test_shape', 0.5, 0.05],['test_shape', 0.7, 0.035], ['test_shape', 1.0, 0.01],
                ['test_shape', 1.1, 0.011]] #,
               #  ['test_scale', 0.1, 0], ['test_scale', 0.25, 0], ['test_scale', 0.5, 0], 
               #   ['test_scale', 0.9, 0]]
# Inv_Gauss_params = [[3, 1, 0.016], [2, 1, 0.025],[1, 1, 0.04]]


In [5]:
# erosion_rates = [0.0992361058410197, 0.09038681072759652, 0.09057589003566213, 0.09160130278196262,
#               0.09264068726487536, 0.09348641593271435, 0.09368159870220641, 0.09664257757763384,
#               0.10308279965901399, 0.10988796191710025, 0.09090216855185158, 0.09206350673112167, 0.09341957264641197, 0.09394608760053381,
#              0.09397132084984372, 0.09431786794189329, 0.09727060627868099, 0.09781215875283383,
#              0.09966615024769897, 0.10803791065764426, 0.09898751248739036, 0.09969268221446984, 0.09978359483916384, 0.10020516727882077,
#              0.10029223342716183, 0.10056328192550705, 0.10069433804927683, 0.10094804059874726,
#              0.10095589942444964, 0.10172451896171934]
# MATs = [0.0, 5.0, -5.0,-10.0, -20.0]
# AMPs = [2.5, 5.0, 10.0, 15.0, 20.0]

In [ ]:
for i in range(len(Pareto_params)):

    %run D:\STEIN_paper\model_outputs\scripts\rockfall_generation_multiple_distributions.ipynb
    rockfall_output = erosion_export(stoch_base_ER, 'Gen_Pareto', Pareto_params[i]) 

    %run D:\STEIN_paper\model_outputs\scripts\C14_Be10_stochastic_erosion.ipynb
    rockfalls_imported = rockfalls_import()
    CRN_output = CRN_loop_fxn(total_time)
    
    write_var_export(i)
    %run D:\STEIN_paper\model_outputs\scripts\He3_stochastic_erosion.ipynb
    He3_output = He3_loop_fxn(total_time)

    %run D:\STEIN_paper\model_outputs\scripts\stochastic_measured_constants_CRN_loops.ipynb
    comparison_constants, samples, measured, measured_ER = compare_fxn()
    og_ER_mat = original_ER_fxn(rockfall_output[1][:])
    master_array, master_df, master_means = post_process(CRN_output, He3_output)
    
    print(datetime.now(), 'Completed Pareto Parameter: ' + str(i))

# for i in range(len(Inv_Gauss_params)):

#     %run D:\STEIN_paper\model_outputs\scripts\rockfall_generation_multiple_distributions.ipynb
#     rockfall_output =  erosion_export(0.1, 'Gen_Pareto', Pareto_params[i])

#     %run D:\STEIN_paper\model_outputs\scripts\C14_Be10_stochastic_erosion.ipynb
#     rockfalls_imported = rockfalls_import()
#     CRN_output = CRN_loop_fxn(total_time)

#     %run D:\STEIN_paper\model_outputs\scripts\He3_stochastic_erosion.ipynb
#     He3_output = He3_loop_fxn(total_time)

#     %run D:\STEIN_paper\model_outputs\scripts\stochastic_measured_constants_CRN_loops.ipynb
#     comparison_constants, samples, measured, measured_ER = compare_fxn()
#     # og_ER_mat = original_ER_fxn(ERs_all)
#     master_array, master_df, master_means = post_process(CRN_output, He3_output)

#     print('Completed Gaussian Parameter: ' + str(i))

#     export_inputs()


In [ ]:
%run D:\STEIN_paper\model_outputs\scripts\C14_Be10_constant_erosion.ipynb
CRN_output = CRN_loop_fxn(total_time, time_ER_shift)

In [ ]:
# for i in range(len(MATs)):    

initial_ER = 0.002
ER_shift_factor = 1.0
%run D:\STEIN_paper\model_outputs\scripts\C14_Be10_constant_erosion.ipynb
CRN_output = CRN_loop_fxn(total_time, time_ER_shift)

%run D:\STEIN_paper\model_outputs\scripts\He3_constant_erosion.ipynb
He3_output = He3_loop_fxn_T_change(total_time, time_ER_shift)
np.savetxt('CON_He3_ER' + str(initial_ER) + '_MAT_' + str(MAT) + '_' + str(T_AMP) +'_surfmat.csv',
               He3_output[1],
               delimiter = ',')


In [ ]:
np.shape(CRN_output[0])

In [ ]:
scaling_factor

In [ ]:
plt.plot(np.divide(He3_output[1],CRN_output[1][0,:])* SLHL_Be10/SLHL_He3)
plt.xscale('log')
plt.xlim([100,100000])
plt.ylim(0,1.0)

In [ ]:
for i in range(len(AMPs)):    
    # %run D:\STEIN_paper\model_outputs\scripts\C14_Be10_constant_erosion.ipynb
    # CRN_output = CRN_loop_fxn(total_time, time_ER_shift)
    initial_ER = 0.0
    MAT = 0.0
    T_AMP = AMPs[i]
    
    %run D:\STEIN_paper\model_outputs\scripts\He3_constant_erosion.ipynb
    He3_output = He3_loop_fxn(total_time, time_ER_shift)
    np.savetxt('CON_He3_ER' + str(initial_ER) + '_MAT_' + str(MAT) + '_' + str(T_AMP) +'_surfmat.csv',
                   He3_output[1],
                   delimiter = ',')
    